In [1]:
import sqlite3
from datetime import datetime, timedelta
import random

# Lists for generating random but realistic data
company_names_prefix = ["Tech", "AI", "Data", "Cloud", "Cyber", "Bio", "Fin", "Health", "Smart", "Green"]
company_names_suffix = ["Systems", "Solutions", "Analytics", "Technologies", "Platform", "Networks", "Labs", "Connect", "Logic", "Scale"]
industries = ["SaaS", "FinTech", "HealthTech", "AI/ML", "Cybersecurity", "BioTech", "CleanTech", "E-commerce", "EdTech", "IoT"]
technologies = ["Python", "Java", "Cloud Native", "Blockchain", "AI/ML", "React", "Mobile", "Big Data", "DevOps", "Kubernetes"]
funding_rounds = ["Seed", "Series A", "Series B", "Series C", "Series D"]
exit_statuses = ["Active", "IPO", "Acquired", "Merged", "Active"]
analysts = [
    "Sarah Johnson", "Michael Chen", "Emma Thompson", "David Kim",
    "Rachel Martinez", "James Wilson", "Lisa Anderson", "Alex Patel",
    "Sarah Johnson", "Michael Chen", "Emma Thompson", "David Kim"  # Some repeated for realistic distribution
]

def random_date(start_year=2018):
    """Generate a random date from start_year to now"""
    start = datetime(start_year, 1, 1)
    end = datetime.now()
    return start + timedelta(
        days=random.randint(0, (end - start).days)
    )

def create_database():
    """Create the database and table"""
    conn = sqlite3.connect('deal_data.db')
    cursor = conn.cursor()
    
    # Create the deals table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS deals (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        company_name TEXT,
        investment_date TEXT,
        industry TEXT,
        technology_stack TEXT,
        funding_round TEXT,
        investment_amount_m REAL,
        revenue_m REAL,
        valuation_m REAL,
        revenue_multiplier REAL,
        equity_stake REAL,
        exit_status TEXT,
        irr_percentage REAL,
        analyst TEXT
    )
    ''')
    
    # Generate and insert sample data
    for _ in range(100):  # Create 100 sample deals
        # Generate company name
        company_name = f"{random.choice(company_names_prefix)}{random.choice(company_names_suffix)}"
        
        # Generate financial data
        revenue = round(random.uniform(1, 100), 2)
        valuation = revenue * random.uniform(5, 20)
        investment_amount = round(random.uniform(1, 50), 2)
        revenue_multiplier = round(valuation / revenue, 2)
        
        # Create deal record
        deal = (
            company_name,
            random_date().strftime('%Y-%m-%d'),
            random.choice(industries),
            random.choice(technologies),
            random.choice(funding_rounds),
            investment_amount,
            revenue,
            round(valuation, 2),
            revenue_multiplier,
            round(random.uniform(5, 30), 2),  # equity_stake
            random.choice(exit_statuses),
            round(random.uniform(15, 100), 2) if random.random() > 0.3 else None,  # irr_percentage
            random.choice(analysts)
        )
        
        # Insert the deal into the database
        cursor.execute('''
        INSERT INTO deals (
            company_name, investment_date, industry, technology_stack,
            funding_round, investment_amount_m, revenue_m, valuation_m,
            revenue_multiplier, equity_stake, exit_status, irr_percentage,
            analyst
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', deal)
    
    # Commit the changes and close the connection
    conn.commit()
    conn.close()

def verify_database():
    """Verify the database contents"""
    conn = sqlite3.connect('deal_data.db')
    cursor = conn.cursor()
    
    # Get all column names
    cursor.execute("PRAGMA table_info(deals)")
    columns = [info[1] for info in cursor.fetchall()]
    
    # Get total number of deals
    cursor.execute("SELECT COUNT(*) FROM deals")
    total_deals = cursor.fetchone()[0]
    
    print(f"\nDatabase created successfully with {total_deals} deals")
    print("\nColumn names:", columns)
    
    # Show sample data
    print("\nSample deals:")
    cursor.execute("SELECT * FROM deals LIMIT 3")
    for deal in cursor.fetchall():
        print("\nDeal:", deal)
    
    # Show analyst distribution
    print("\nDeals per analyst:")
    cursor.execute("""
    SELECT analyst, COUNT(*) as deal_count 
    FROM deals 
    GROUP BY analyst 
    ORDER BY deal_count DESC
    """)
    for analyst, count in cursor.fetchall():
        print(f"{analyst}: {count} deals")
    
    conn.close()

if __name__ == "__main__":
    try:
        create_database()
        verify_database()
        
        print("\nYou can now query the database using SQL, for example:")
        print("""
        conn = sqlite3.connect('deal_data.db')
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM deals WHERE industry = 'FinTech'")
        fintech_deals = cursor.fetchall()
        """)
        
    except Exception as e:
        print(f"Error: {str(e)}")


Database created successfully with 100 deals

Column names: ['id', 'company_name', 'investment_date', 'industry', 'technology_stack', 'funding_round', 'investment_amount_m', 'revenue_m', 'valuation_m', 'revenue_multiplier', 'equity_stake', 'exit_status', 'irr_percentage', 'analyst']

Sample deals:

Deal: (1, 'TechLabs', '2024-08-11', 'HealthTech', 'Cloud Native', 'Series D', 16.41, 69.81, 1278.87, 18.32, 8.39, 'Merged', 95.08, 'Alex Patel')

Deal: (2, 'CyberNetworks', '2021-03-24', 'SaaS', 'Kubernetes', 'Series D', 26.26, 39.46, 335.22, 8.5, 18.04, 'Merged', 93.4, 'Michael Chen')

Deal: (3, 'AIScale', '2024-08-16', 'FinTech', 'React', 'Seed', 27.91, 23.4, 433.52, 18.53, 17.01, 'Acquired', 75.62, 'James Wilson')

Deals per analyst:
Michael Chen: 24 deals
Emma Thompson: 15 deals
Sarah Johnson: 13 deals
David Kim: 13 deals
Alex Patel: 11 deals
Rachel Martinez: 8 deals
Lisa Anderson: 8 deals
James Wilson: 8 deals

You can now query the database using SQL, for example:

        conn = sqli